In [1]:
import pandas as pd
import numpy as np
import requests
import os
import time
# import matplotlib.pyplot as plt

In [2]:
token = "dc76ceadd678dd146f345afcb88619b4ab0287e9"
headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Token ' + token
        }
requests.get("https://api.tiingo.com/api/test/",
                                    headers=headers)

<Response [200]>

In [4]:
import time
from datetime import datetime as dt
from datetime import timedelta

_from = dt.now()
to = _from + timedelta(hours=10, minutes=5)

In [5]:
class counter:
    def __init__(self, wait_h, wait_m, wait_s, period_max):
        self.wait_h = wait_h
        self.wait_m = wait_m
        self.wait_s = wait_s
        self.wait = timedelta(hours=self.wait_h, minutes=self.wait_m, seconds=self.wait_s)
        
        self.period_max = period_max
        
        self.start = dt.now()
        self.to = self.start + self.wait
        self.count = 0
        
    def add(self):
        self.last_add = dt.now()
        if self.last_add > self.start+self.wait:
            self.reset()
        
        self.count+=1
        
        if self.count == self.period_max:
            print('Reach maximum request in a period, wait until %s' %self.to)
            time.sleep((self.to - dt.now()).seconds)
            self.reset()
   
    def reset(self):
        self.start = dt.now()
        self.to = self.start +  self.wait
        self.count = 0
        print('counter reset')

In [6]:
_counter = counter(0,0,3,20)

for i in range(500):
    _counter.add()
    print(_counter.count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum request in a period, wait until 2021-07-16 19:46:07.188908
counter reset
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Reach maximum request in a period, wait until 2021-07-16 19:46:09.193789


KeyboardInterrupt: 

In [153]:
_counter.add()

counter reset


In [7]:
def get_price(tickers, dates, dates_end, _counter):
    error_log = []

    for d1, d2 in zip(dates, dates_end):
        d1 = str(d1).split(' ')[0]
        d2 = str(d2).split(' ')[0]
        print(d1, d2)

        query = {'tickers':tickers,
                 'startDate': d1, 
                 'endDate': d2,
                 'resampleFreq': '1hour'}

        
        r = requests.get("https://api.tiingo.com/tiingo/crypto/prices", 
                         params = query,
                         headers=headers)
        
        _counter.add()
        print('hourly request count:',_counter.count)

        if len(r.json())==0:
            error_log.append([(tickers, d1,d2)])
            print('Empty Json')

        else:
            df=pd.DataFrame(r.json()[0]['priceData'])
            df=df[['date', 'open', 'high', 'low', 'close', 'volume', 'tradesDone', 'volumeNotional']]
            df.to_csv(filename, mode="a", header=False)
        
    return error_log
#     time.sleep(0.5)          

In [8]:
def clean_data(filename):
    file = pd.read_csv(filename, header=None)
    file.columns = ['id', 'date', 'open', 'high', 'low', 'close', 'volume', 'tradesDone', 'volumeNotional']

    file.drop_duplicates('date', inplace=True)

    file.date = pd.to_datetime(file.date)
#     file = file.loc[:,['date', 'open']]
#     file.columns = ['Date', '%s Price'%coin]
    file.to_csv(filename)

In [20]:
coins = pd.read_pickle('CCI30_data_request.pkl')
coins

,ticker,start,end
0,aave,2020-10-01,2021-08-01
1,ae,2018-07-01,2018-10-02
2,agrs,2017-01-01,2017-04-01
3,algo,2021-07-01,2021-08-01
4,amp,2021-07-01,2021-08-01
...,...,...,...
75,xvg,2018-01-01,2018-07-02
76,yfi,2020-10-01,2021-01-01
77,zec,2017-01-01,2021-01-01
78,zil,2018-07-01,2019-04-01


In [23]:
for coin in coins:
    print(coin)

ticker
start
end


In [18]:
coin = coins.iloc[0]
pd.date_range(start=coin.start, end=coin.end, freq="3D")


coin.start + timedelta(days=3)

Timestamp('2020-10-04 00:00:00')

In [25]:
coins.iloc[0,:].ticker

'aave'

In [26]:
# coins = ['ada', 'bch', 'bnb', 'bsv', 'btc', 'dot', 'eos', 'eth', 'link',
#        'ltc', 'xrp']

# coins = ['bsv', 'btc', 'dot', 'eos', 'eth', 'link',
#        'ltc', 'xrp']

# coins = ['xrp', 'eth']
# coins = ['xem', 'gam', 'dash', 'etc', 'xmr']
# coins = ['bnb']



_counter = counter(1,0,3, 450)
error_log = []
error_tick = []

for r in range(len(coins)):
    coin = coins.iloc[r,:]
    unit = 'usd'
    tickers = coin.ticker+unit
    filename="tiingo_%s.csv"%coin

    print(filename)
    print(tickers)
    
    
    start1 = coin.start
    start2 = start1 + timedelta(days=3)
    
    end1 = coin.end
    end2 = end1 + timedelta(days=3)
    
    dates = pd.date_range(start=start1, end=end1, freq="3D")
    dates_end = pd.date_range(start=start2, end=end2, freq="3D")

    
    error = get_price(tickers, dates, dates_end, _counter)
    error_log.append(error)
    try:
        clean_data(filename)
    except:
        error_tick.append(coin)


tiingo_ticker                   aave
start     2020-10-01 00:00:00
end       2021-08-01 00:00:00
Name: 0, dtype: object.csv
aaveusd
2020-10-01 2020-10-04
hourly request count: 1
Empty Json
2020-10-04 2020-10-07
hourly request count: 2
Empty Json
2020-10-07 2020-10-10
hourly request count: 3
Empty Json
2020-10-10 2020-10-13
hourly request count: 4
Empty Json
2020-10-13 2020-10-16
hourly request count: 5
Empty Json
2020-10-16 2020-10-19
hourly request count: 6
Empty Json
2020-10-19 2020-10-22
hourly request count: 7
Empty Json
2020-10-22 2020-10-25
hourly request count: 8
Empty Json
2020-10-25 2020-10-28
hourly request count: 9
2020-10-28 2020-10-31
hourly request count: 10
2020-10-31 2020-11-03
hourly request count: 11
2020-11-03 2020-11-06
hourly request count: 12
2020-11-06 2020-11-09
hourly request count: 13
2020-11-09 2020-11-12
hourly request count: 14
2020-11-12 2020-11-15
hourly request count: 15
2020-11-15 2020-11-18
hourly request count: 16
2020-11-18 2020-11-21
hourly request 

hourly request count: 152
Empty Json
2017-02-24 2017-02-27
hourly request count: 153
Empty Json
2017-02-27 2017-03-02
hourly request count: 154
Empty Json
2017-03-02 2017-03-05
hourly request count: 155
Empty Json
2017-03-05 2017-03-08
hourly request count: 156
Empty Json
2017-03-08 2017-03-11
hourly request count: 157
Empty Json
2017-03-11 2017-03-14
hourly request count: 158
Empty Json
2017-03-14 2017-03-17
hourly request count: 159
Empty Json
2017-03-17 2017-03-20
hourly request count: 160
Empty Json
2017-03-20 2017-03-23
hourly request count: 161
Empty Json
2017-03-23 2017-03-26
hourly request count: 162
Empty Json
2017-03-26 2017-03-29
hourly request count: 163
Empty Json
2017-03-29 2017-04-01
hourly request count: 164
Empty Json
2017-04-01 2017-04-04
hourly request count: 165
Empty Json
tiingo_ticker                   algo
start     2021-07-01 00:00:00
end       2021-08-01 00:00:00
Name: 3, dtype: object.csv
algousd
2021-07-01 2021-07-04
hourly request count: 166
2021-07-04 2021-

hourly request count: 287
Empty Json
2017-10-28 2017-10-31
hourly request count: 288
Empty Json
2017-10-31 2017-11-03
hourly request count: 289
Empty Json
2017-11-03 2017-11-06
hourly request count: 290
Empty Json
2017-11-06 2017-11-09
hourly request count: 291
Empty Json
2017-11-09 2017-11-12
hourly request count: 292
Empty Json
2017-11-12 2017-11-15
hourly request count: 293
Empty Json
2017-11-15 2017-11-18
hourly request count: 294
Empty Json
2017-11-18 2017-11-21
hourly request count: 295
Empty Json
2017-11-21 2017-11-24
hourly request count: 296
Empty Json
2017-11-24 2017-11-27
hourly request count: 297
Empty Json
2017-11-27 2017-11-30
hourly request count: 298
Empty Json
2017-11-30 2017-12-03
hourly request count: 299
Empty Json
2017-12-03 2017-12-06
hourly request count: 300
Empty Json
2017-12-06 2017-12-09
hourly request count: 301
Empty Json
2017-12-09 2017-12-12
hourly request count: 302
Empty Json
2017-12-12 2017-12-15
hourly request count: 303
Empty Json
2017-12-15 2017-12-

hourly request count: 433
2020-01-06 2020-01-09
hourly request count: 434
2020-01-09 2020-01-12
hourly request count: 435
2020-01-12 2020-01-15
hourly request count: 436
2020-01-15 2020-01-18
hourly request count: 437
2020-01-18 2020-01-21
hourly request count: 438
2020-01-21 2020-01-24
hourly request count: 439
2020-01-24 2020-01-27
hourly request count: 440
2020-01-27 2020-01-30
hourly request count: 441
2020-01-30 2020-02-02
hourly request count: 442
2020-02-02 2020-02-05
hourly request count: 443
2020-02-05 2020-02-08
hourly request count: 444
2020-02-08 2020-02-11
hourly request count: 445
2020-02-11 2020-02-14
hourly request count: 446
2020-02-14 2020-02-17
hourly request count: 447
2020-02-17 2020-02-20
hourly request count: 448
2020-02-20 2020-02-23
hourly request count: 449
2020-02-23 2020-02-26
Reach maximum request in a period, wait until 2021-07-16 20:53:17.679476
counter reset
hourly request count: 0
2020-02-26 2020-02-29
hourly request count: 1
2020-02-29 2020-03-03
hourl

hourly request count: 155
2021-06-05 2021-06-08
hourly request count: 156
2021-06-08 2021-06-11
hourly request count: 157
2021-06-11 2021-06-14
hourly request count: 158
2021-06-14 2021-06-17
hourly request count: 159
2021-06-17 2021-06-20
hourly request count: 160
2021-06-20 2021-06-23
hourly request count: 161
2021-06-23 2021-06-26
hourly request count: 162
2021-06-26 2021-06-29
hourly request count: 163
2021-06-29 2021-07-02
hourly request count: 164
2021-07-02 2021-07-05
hourly request count: 165
tiingo_ticker                   avax
start     2021-04-01 00:00:00
end       2021-07-02 00:00:00
Name: 8, dtype: object.csv
avaxusd
2021-04-01 2021-04-04
hourly request count: 166
Empty Json
2021-04-04 2021-04-07
hourly request count: 167
Empty Json
2021-04-07 2021-04-10
hourly request count: 168
Empty Json
2021-04-10 2021-04-13
hourly request count: 169
Empty Json
2021-04-13 2021-04-16
hourly request count: 170
Empty Json
2021-04-16 2021-04-19
hourly request count: 171
Empty Json
2021-04-

...

hourly request count: 412
Empty Json
2017-09-07 2017-09-10
hourly request count: 413
Empty Json
2017-09-10 2017-09-13
hourly request count: 414
Empty Json
2017-09-13 2017-09-16
hourly request count: 415
Empty Json
2017-09-16 2017-09-19
hourly request count: 416
Empty Json
2017-09-19 2017-09-22
hourly request count: 417
Empty Json
2017-09-22 2017-09-25
hourly request count: 418
Empty Json
2017-09-25 2017-09-28
hourly request count: 419
Empty Json
2017-09-28 2017-10-01
hourly request count: 420
Empty Json
2017-10-01 2017-10-04
hourly request count: 421
Empty Json
2017-10-04 2017-10-07
hourly request count: 422
Empty Json
2017-10-07 2017-10-10
hourly request count: 423
Empty Json
2017-10-10 2017-10-13
hourly request count: 424
Empty Json
2017-10-13 2017-10-16
hourly request count: 425
Empty Json
2017-10-16 2017-10-19
hourly request count: 426
Empty Json
2017-10-19 2017-10-22
hourly request count: 427
Empty Json
2017-10-22 2017-10-25
hourly request count: 428
Empty Json
2017-10-25 2017-10-

hourly request count: 106
2018-11-13 2018-11-16
hourly request count: 107
2018-11-16 2018-11-19
hourly request count: 108
2018-11-19 2018-11-22
hourly request count: 109
2018-11-22 2018-11-25
hourly request count: 110
2018-11-25 2018-11-28
hourly request count: 111
2018-11-28 2018-12-01
hourly request count: 112
2018-12-01 2018-12-04
hourly request count: 113
2018-12-04 2018-12-07
hourly request count: 114
2018-12-07 2018-12-10
hourly request count: 115
2018-12-10 2018-12-13
hourly request count: 116
2018-12-13 2018-12-16
hourly request count: 117
2018-12-16 2018-12-19
hourly request count: 118
2018-12-19 2018-12-22
hourly request count: 119
2018-12-22 2018-12-25
hourly request count: 120
2018-12-25 2018-12-28
hourly request count: 121
2018-12-28 2018-12-31
hourly request count: 122
2018-12-31 2019-01-03
hourly request count: 123
2019-01-03 2019-01-06
hourly request count: 124
2019-01-06 2019-01-09
hourly request count: 125
2019-01-09 2019-01-12
hourly request count: 126
2019-01-12 201

hourly request count: 276
2020-04-06 2020-04-09
hourly request count: 277
2020-04-09 2020-04-12
hourly request count: 278
2020-04-12 2020-04-15
hourly request count: 279
2020-04-15 2020-04-18
hourly request count: 280
2020-04-18 2020-04-21
hourly request count: 281
2020-04-21 2020-04-24
hourly request count: 282
2020-04-24 2020-04-27
hourly request count: 283
2020-04-27 2020-04-30
hourly request count: 284
2020-04-30 2020-05-03
hourly request count: 285
2020-05-03 2020-05-06
hourly request count: 286
2020-05-06 2020-05-09
hourly request count: 287
2020-05-09 2020-05-12
hourly request count: 288
2020-05-12 2020-05-15
hourly request count: 289
2020-05-15 2020-05-18
hourly request count: 290
2020-05-18 2020-05-21
hourly request count: 291
2020-05-21 2020-05-24
hourly request count: 292
2020-05-24 2020-05-27
hourly request count: 293
2020-05-27 2020-05-30
hourly request count: 294
2020-05-30 2020-06-02
hourly request count: 295
2020-06-02 2020-06-05
hourly request count: 296
2020-06-05 202

hourly request count: 442
Empty Json
2017-01-16 2017-01-19
hourly request count: 443
Empty Json
2017-01-19 2017-01-22
hourly request count: 444
Empty Json
2017-01-22 2017-01-25
hourly request count: 445
Empty Json
2017-01-25 2017-01-28
hourly request count: 446
Empty Json
2017-01-28 2017-01-31
hourly request count: 447
Empty Json
2017-01-31 2017-02-03
hourly request count: 448
Empty Json
2017-02-03 2017-02-06
hourly request count: 449
Empty Json
2017-02-06 2017-02-09
Reach maximum request in a period, wait until 2021-07-17 01:53:30.204630
counter reset
hourly request count: 0
Empty Json
2017-02-09 2017-02-12
hourly request count: 1
Empty Json
2017-02-12 2017-02-15
hourly request count: 2
Empty Json
2017-02-15 2017-02-18
hourly request count: 3
Empty Json
2017-02-18 2017-02-21
hourly request count: 4
Empty Json
2017-02-21 2017-02-24
hourly request count: 5
Empty Json
2017-02-24 2017-02-27
hourly request count: 6
Empty Json
2017-02-27 2017-03-02
hourly request count: 7
Empty Json
2017-03

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [27]:
error_log

[[[('aaveusd', '2020-10-01', '2020-10-04')],
  [('aaveusd', '2020-10-04', '2020-10-07')],
  [('aaveusd', '2020-10-07', '2020-10-10')],
  [('aaveusd', '2020-10-10', '2020-10-13')],
  [('aaveusd', '2020-10-13', '2020-10-16')],
  [('aaveusd', '2020-10-16', '2020-10-19')],
  [('aaveusd', '2020-10-19', '2020-10-22')],
  [('aaveusd', '2020-10-22', '2020-10-25')],
  [('aaveusd', '2021-07-19', '2021-07-22')],
  [('aaveusd', '2021-07-22', '2021-07-25')],
  [('aaveusd', '2021-07-25', '2021-07-28')],
  [('aaveusd', '2021-07-28', '2021-07-31')],
  [('aaveusd', '2021-07-31', '2021-08-03')]],
 [[('aeusd', '2018-07-01', '2018-07-04')],
  [('aeusd', '2018-07-04', '2018-07-07')],
  [('aeusd', '2018-07-07', '2018-07-10')],
  [('aeusd', '2018-07-10', '2018-07-13')],
  [('aeusd', '2018-07-13', '2018-07-16')],
  [('aeusd', '2018-07-16', '2018-07-19')],
  [('aeusd', '2018-07-19', '2018-07-22')],
  [('aeusd', '2018-07-22', '2018-07-25')],
  [('aeusd', '2018-07-25', '2018-07-28')],
  [('aeusd', '2018-07-28', 

In [28]:
error_tick

[ticker                     ae
 start     2018-07-01 00:00:00
 end       2018-10-02 00:00:00
 Name: 1, dtype: object, ticker                   agrs
 start     2017-01-01 00:00:00
 end       2017-04-01 00:00:00
 Name: 2, dtype: object, ticker                   ardr
 start     2017-01-01 00:00:00
 end       2018-04-01 00:00:00
 Name: 5, dtype: object, ticker                    ark
 start     2017-10-01 00:00:00
 end       2018-01-01 00:00:00
 Name: 6, dtype: object, ticker                   bccx
 start     2017-04-01 00:00:00
 end       2018-02-01 00:00:00
 Name: 9, dtype: object, ticker                    btm
 start     2018-07-01 00:00:00
 end       2018-10-02 00:00:00
 Name: 12, dtype: object, ticker                    bts
 start     2017-01-01 00:00:00
 end       2018-02-01 00:00:00
 Name: 13, dtype: object, ticker                    cel
 start     2021-01-01 00:00:00
 end       2021-04-01 00:00:00
 Name: 16, dtype: object, ticker                    dgd
 start     2017-01-01 00:00:00

In [119]:
error_tick

# bnbusdt, usdtusd

error = get_price('bnbusdt', dates, dates_end, _counter)
filename="tiingo_bnbusdt.csv"
clean_data(filename)


2017-12-15 2017-12-18
hourly request count: 451
Empty Json
2017-12-18 2017-12-21
hourly request count: 452
Empty Json
2017-12-21 2017-12-24
hourly request count: 453
Empty Json
2017-12-24 2017-12-27
hourly request count: 454
Empty Json
2017-12-27 2017-12-30
hourly request count: 455
Empty Json
2017-12-30 2018-01-02
hourly request count: 456
Empty Json
2018-01-02 2018-01-05
hourly request count: 457
Empty Json
2018-01-05 2018-01-08
hourly request count: 458
Empty Json
2018-01-08 2018-01-11
hourly request count: 459
Empty Json
2018-01-11 2018-01-14
hourly request count: 460
Empty Json
2018-01-14 2018-01-17
hourly request count: 461
Empty Json
2018-01-17 2018-01-20
hourly request count: 462
Empty Json
2018-01-20 2018-01-23
hourly request count: 463
Empty Json
2018-01-23 2018-01-26
hourly request count: 464
Empty Json
2018-01-26 2018-01-29
hourly request count: 465
Empty Json
2018-01-29 2018-02-01
hourly request count: 466
Empty Json
2018-02-01 2018-02-04
hourly request count: 467
Empty Js

hourly request count: 590
Empty Json
2019-02-08 2019-02-11
hourly request count: 591
Empty Json
2019-02-11 2019-02-14
hourly request count: 592
Empty Json
2019-02-14 2019-02-17
hourly request count: 593
Empty Json
2019-02-17 2019-02-20
hourly request count: 594
Empty Json
2019-02-20 2019-02-23
hourly request count: 595
Empty Json
2019-02-23 2019-02-26
hourly request count: 596
Empty Json
2019-02-26 2019-03-01
hourly request count: 597
Empty Json
2019-03-01 2019-03-04
hourly request count: 598
Empty Json
2019-03-04 2019-03-07
hourly request count: 599
Empty Json
2019-03-07 2019-03-10
hourly request count: 600
Empty Json
2019-03-10 2019-03-13
hourly request count: 601
Empty Json
2019-03-13 2019-03-16
hourly request count: 602
Empty Json
2019-03-16 2019-03-19
hourly request count: 603
Empty Json
2019-03-19 2019-03-22
hourly request count: 604
Empty Json
2019-03-22 2019-03-25
hourly request count: 605
Empty Json
2019-03-25 2019-03-28
hourly request count: 606
Empty Json
2019-03-28 2019-03-

hourly request count: 729
Empty Json
2020-03-31 2020-04-03
hourly request count: 730
Empty Json
2020-04-03 2020-04-06
hourly request count: 731
Empty Json
2020-04-06 2020-04-09
hourly request count: 732
Empty Json
2020-04-09 2020-04-12
hourly request count: 733
Empty Json
2020-04-12 2020-04-15
hourly request count: 734
Empty Json
2020-04-15 2020-04-18
hourly request count: 735
Empty Json
2020-04-18 2020-04-21
hourly request count: 736
Empty Json
2020-04-21 2020-04-24
hourly request count: 737
Empty Json
2020-04-24 2020-04-27
hourly request count: 738
Empty Json
2020-04-27 2020-04-30
hourly request count: 739
Empty Json
2020-04-30 2020-05-03
hourly request count: 740
Empty Json
2020-05-03 2020-05-06
hourly request count: 741
Empty Json
2020-05-06 2020-05-09
hourly request count: 742
Empty Json
2020-05-09 2020-05-12
hourly request count: 743
Empty Json
2020-05-12 2020-05-15
hourly request count: 744
Empty Json
2020-05-15 2020-05-18
hourly request count: 745
Empty Json
2020-05-18 2020-05-

hourly request count: 870
2021-05-28 2021-05-31
hourly request count: 871
2021-05-31 2021-06-03
hourly request count: 872


FileNotFoundError: [Errno 2] No such file or directory: 'tiingo_bnbusdt.csv'

In [ ]:
error = get_price('usdtusd', dates, dates_end, _counter)
filename="tiingo_usdtusd.csv"
clean_data(filename)

In [163]:
tickers = 'bnbusdt'
query = {'tickers':tickers,
         'startDate': "2021-05-04", 
         'endDate': "2021-06-04",
         'resampleFreq': '1Day'}

r = requests.get("https://api.tiingo.com/tiingo/crypto/prices", 
                 params = query,
                 headers=headers)

In [165]:
r.json()

[{'ticker': 'bnbusdt',
  'baseCurrency': 'bnb',
  'quoteCurrency': 'usdt',
  'priceData': [{'tradesDone': 2253.0,
    'date': '2021-05-12T00:00:00+00:00',
    'close': 589.53768,
    'volumeNotional': 878139.9558672,
    'high': 684.71075,
    'low': 589.53768,
    'open': 672.32682,
    'volume': 1489.54},
   {'tradesDone': 2135.0,
    'date': '2021-05-13T00:00:00+00:00',
    'close': 571.67343,
    'volumeNotional': 957604.4458587001,
    'high': 632.72353,
    'low': 535.04338,
    'open': 572.21528,
    'volume': 1675.09},
   {'tradesDone': 1048.0,
    'date': '2021-05-14T00:00:00+00:00',
    'close': 602.52213,
    'volumeNotional': 383999.40389159997,
    'high': 614.30499,
    'low': 566.96852,
    'open': 568.11593,
    'volume': 637.32},
   {'tradesDone': 889.0,
    'date': '2021-05-15T00:00:00+00:00',
    'close': 563.42771,
    'volumeNotional': 433546.35429080005,
    'high': 610.55551,
    'low': 561.17258,
    'open': 605.923,
    'volume': 769.48},
   {'tradesDone': 1408